# Assignment 3  
## Applied Machine Learning

Andrew Chan 
EBE869

In [119]:
import pandas as pd
import math
import numpy as np

# Locate and load the data file
df = pd.read_csv('master.csv')

# Sanity check
print(f'N rows={len(df)}, M columns={len(df.columns)}')
df.head()

N rows=27820, M columns=12


,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [120]:
df.columns.values

array(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype=object)

# 1. [20 pts] Due to the severity of this real-world crisis, what information would be the most important one to "machine learn"? Can it be learned?

The most important target variable to machine learn is `suicides_no`. To attempt a machine learning solution we will turn this into a binary classification problem of `low suicide rate` versus `high suicide rate`. Framing the problem this way will allow us to train a classifier. 

# 2. [20 pts] Explain in detail how one should set up the problem? Would it be a regression or a classification problem? Any unsupervised approach to see some patterns exist or not?

We will frame this as a classification problem with `low suicide rate` and `high suicide rate`. 

# 3. [20 pts] What should be the dependent variable?

The dependent variable will be generated from the `suicides_no`.

In [123]:
suicides_mean = df["suicides_no"].mean()
suicides_mean

242.57440690150972

We will create a new dependent variable called `suicides_rate` that is derived from `suicides_no` as follows:  
* All values larger than or equal to `suicides_mean` will be labelled with `high suicide rate` 
* All values smaller than `suicides_mean` will be labelled with `low suicide rate` 

In [131]:
def isHigh(x, mean_val):
    if x > mean_val:
        return True
    else:
        return False

In [136]:
modDfObj = df['suicides_no'].apply(isHigh, args=[suicides_mean])

In [137]:
modDfObj

0        False
1        False
2        False
3        False
4        False
         ...  
27815    False
27816    False
27817    False
27818    False
27819    False
Name: suicides_no, Length: 27820, dtype: bool

In [139]:
type(modDfObj)

pandas.core.series.Series

In [143]:
modDfObj = modDfObj.rename('suicides_rate')

In [ ]:
df.append(modDfObj, ignore_index=True)

# 4. [20 pts] Rank the variables to find some strong correlations between the independent variables and the dependent variable you decided

# 5. [20 pts] Pre-process the dataset and list the major features you want to use. 
Note that not all features are crucial. For example, country-year variable is a derived feature and for a classifier it would not be necessary to include the year, the country and the country-year together. In fact, one must avoid adding a derived feature and the original at the same time.

List the independent features you want to use.

# 6. [20 pts bonus] Devise a classification problem and present a prototype model.
Note that we will continue to this problem in the following modules.